In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
from sklearn.preprocessing import LabelEncoder

In [33]:
import importlib
import Model_functions
importlib.reload(Model_functions)
from Model_functions import Model_class 

In [3]:
df = pd.read_csv("df_127_columns.csv")

In [3]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]
reduce_memory = low_cardinality_cat_cols + ["mode_size","mode_item_color"]

In [6]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531170 entries, 0 to 531169
Columns: 127 entries, order_item_id to delivery_time_user_id_mad
dtypes: float64(54), int64(62), object(11)
memory usage: 825.7 MB


It seems that quite a few columns are stored as object types which use more memory than necessary. We can convert these to the appropriate types to save memory as we will use a quite substantial amount of feature engineering which will increase the memory requirements of storing the dataset in RAM.


In [6]:
# Convert order_id to category and the datetime features to a format YYYY-MM-DD
df["order_id"] = df["order_id"].astype("category")
df["order_date"] = pd.to_datetime(df["order_date"], format="%Y-%m-%d")
df["delivery_date"] = pd.to_datetime(df["delivery_date"], format="%Y-%m-%d")
df["user_reg_date"] = pd.to_datetime(df["user_reg_date"], format="%Y-%m-%d")
df["user_dob"] = pd.to_datetime(df["user_dob"], format="%Y-%m-%d")

# Use label encoder to convert the categorical features to numeric which will change the column type to int and reduce memory
LE = LabelEncoder()
for i in reduce_memory:
    df[i] = LE.fit_transform(df[i])

In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531170 entries, 0 to 531169
Columns: 127 entries, order_item_id to delivery_time_user_id_mad
dtypes: category(1), datetime64[ns](4), float64(54), int32(6), int64(62)
memory usage: 515.1 MB


As seen from the above output we managed to reduce the memory usage of the dataframe from 825 MB to 515 MB which constitutes a reduction of **37.5%** in memory usage.

In [9]:
# How to shift the date
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)

In the next steps, we will be deriving new time-based features from the order and delivery dates in the hopes they will be useful in predicting the customer's return rate. First we are going to start with the days since the last order for each customer. We can hypothesize that if a customer has placed two orders in a short period of time, they are more likely to return an item.

In [10]:
# Get days since last order for each user
df_group = df[["order_date","user_id"]].drop_duplicates().sort_values(by=["user_id","order_date"])
df_group["order_date_shifted"] = df_group.groupby("user_id")["order_date"].shift(1)
df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
df_group

,order_date,user_id,order_date_shifted,days_since_last_order
24723,2012-04-19,6,NaT,NaN
31011,2012-04-25,6,2012-04-19,6.0
226242,2012-09-23,6,2012-04-25,151.0
240276,2012-10-06,6,2012-09-23,13.0
331324,2013-01-11,6,2012-10-06,97.0
...,...,...,...,...
531141,2013-04-29,91911,NaT,NaN
531149,2013-04-27,91912,NaT,NaN
531154,2013-04-29,91915,NaT,NaN
531158,2013-04-29,91920,NaT,NaN


In [11]:
# Get statistics for days since last order
cols_to_agg = ["user_id"]
stats_to_agg = ["mean","median","max","min","std","mad","skew"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group_group = df_group[[i,"days_since_last_order"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group_group.columns = [str("_" + i +"_").join(col) for col in df_group_group.columns]
    df_group_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group = df_group.merge(df_group_group,on=i,how="left")
df_group.fillna(0,inplace=True)
df_group.drop(["order_date_shifted"],axis=1,inplace=True)
df = df.merge(df_group,on=["user_id","order_date"],how="left")    

In [20]:
# Create a powerset of the item descriptions columns
item_descriptions = ["item_id","size","item_color","brand_id"]
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

Furthermore we will also be deriving the days since the last order for each customer for each product and every product variation such as size and color, and for each brand, as well as their interactions.

In [13]:
categorical_powerset = powerset(item_descriptions)
# Use the powerset to create new columns indicating the days since the last order for each combination of the item descriptions
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude):
        print(current)
        df_group = df[["order_date","user_id"]+current].drop_duplicates().sort_values(by=["user_id","order_date"])
        df_group["order_date_shifted"] = df_group.groupby(["user_id"] + current)["order_date"].shift(1)
        df_group["days_since_last_order_same"+ "_" + "_".join(current)] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
        df_group.fillna(0, inplace=True)
        df_group.drop(["order_date_shifted"],axis=1,inplace=True)
        df = df.merge(df_group,on=["user_id","order_date"]+current,how="left")

['item_id']
['size']
['item_color']
['brand_id']
['item_id', 'size']
['item_id', 'item_color']
['size', 'item_color']
['size', 'brand_id']
['item_color', 'brand_id']
['item_id', 'size', 'item_color']
['size', 'item_color', 'brand_id']


In [14]:
# Get days since last delivery
df_group = df[["delivery_date","user_id"]].drop_duplicates().sort_values(by=["user_id","delivery_date"])
df_group["delivery_date_shifted"] = df_group.groupby("user_id")["delivery_date"].shift(1)
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)
df_group["days_since_last_delivery"] = (df_group["delivery_date"] - df_group["delivery_date_shifted"]).dt.days
# df_group.fillna(0, inplace=True)
df_group

,delivery_date,user_id,delivery_date_shifted,days_since_last_delivery
24723,2012-04-24,6,NaT,NaN
31011,2012-04-26,6,2012-04-24,2.0
31013,2012-05-07,6,2012-04-26,11.0
226242,2012-09-27,6,2012-05-07,143.0
240281,2012-10-10,6,2012-09-27,13.0
...,...,...,...,...
531146,2013-05-20,91911,2013-05-03,17.0
531149,2013-06-10,91912,NaT,NaN
531154,2013-05-03,91915,NaT,NaN
531158,2013-05-03,91920,NaT,NaN


Repeat the same procedure for the delivery dates.

In [15]:
# get statistics for days since last delivery
cols_to_agg = ["user_id"]
stats_to_agg = ["mean","median","max","min","std","mad","skew"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group_group = df_group[[i,"days_since_last_delivery"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group_group.columns = [str("_" + i +"_").join(col) for col in df_group_group.columns]
    df_group_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group = df_group.merge(df_group_group,on=i,how="left")
df_group.fillna(0,inplace=True)
df_group.drop(["delivery_date_shifted"],axis=1,inplace=True)
df = df.merge(df_group,on=["user_id","delivery_date"],how="left")   

In [17]:
categorical_powerset = powerset(item_descriptions)
# Use the powerset to create new columns indicating the days since the last delivery for each combination of the item descriptions
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude):
        print(current)
        df_group = df[["delivery_date","user_id"]+current].drop_duplicates().sort_values(by=["user_id","delivery_date"])
        df_group["delivery_date_shifted"] = df_group.groupby(["user_id"] + current)["delivery_date"].shift(1)
        df_group["days_since_last_delivery_same"+ "_" + "_".join(current)] = (df_group["delivery_date"] - df_group["delivery_date_shifted"]).dt.days
        df_group.fillna(0, inplace=True)
        df_group.drop(["delivery_date_shifted"],axis=1,inplace=True)
        df = df.merge(df_group,on=["user_id","delivery_date"]+current,how="left")

['item_id']
['size']
['item_color']
['brand_id']
['item_id', 'size']
['item_id', 'item_color']
['size', 'item_color']
['size', 'brand_id']
['item_color', 'brand_id']
['item_id', 'size', 'item_color']
['size', 'item_color', 'brand_id']


To get the summary statistics (mean, max, min...) for the interactions between the numeric variable **delivery date** and other numeric variables such as **order_sum** and **item_price** we will bin these variables into different bins and then calculate the summary statistics for each bin.

In [19]:
# Use the numeric variable order_sum to create a new categorical variable using bins
# Bins for the order_sum variable
bins_range = df["order_sum"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])
bins = bins_range[["min","10%","25%","50%","75%","90%","max"]].values
df['order_sum_bins'] = pd.cut(df['order_sum'], bins=bins, include_lowest=True).cat.codes

# Bins for the item_price variable
bins_range = df["item_price"].describe(percentiles=[0.1,0.25,0.5,0.75,0.95])
bins = bins_range[["min","10%","25%","50%","75%","95%","max"]].values
df['item_price_bins'] = pd.cut(df['item_price'], bins=bins, include_lowest=True).cat.codes

In [20]:
# Bins for the age variable based on the life stages of the customer
bins = [20,24,29,34,44,55,65,85]
df['age_bins'] = pd.cut(df['user_age'], bins=bins, include_lowest=True).cat.codes

In [21]:
df["user_reg_age"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])
bins = [-1,0,0.1,7,14,30,60,90,180,365,730,805]
df['reg_age_bins'] = pd.cut(df['user_reg_age'], bins=bins, include_lowest=True).cat.codes

In [22]:
bins_column_list = ["order_sum_bins","item_price_bins","age_bins","reg_age_bins"]

In [23]:
# Get the interaction between the bins, the order/delivery weekday and the delivery time
cols_to_agg = bins_column_list + ["order_weekday", "delivery_weekday"]
stats_to_agg = ["mean","std","min","max","median","mad","skew"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group = df[[i,"delivery_time"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group.columns = [str("_" + i +"_").join(col) for col in df_group.columns]
    df_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group.fillna(0,inplace=True)
    df = df.merge(df_group,on=i,how="left")   

In [24]:
df.drop(bins_column_list,axis=1,inplace=True)

In [7]:
def powerset(iterable,r1,r2):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(r1,r2))

Also, add the interaction between the **delivery date** and the categorical variables including the user by capturing the summary statistics for each category and the interaction between categories

In [ ]:
# Interactions between delivery time and categorical variables
interactions_list = ["item_id","brand_id","item_color","size","delivery_weekday","user_id"]
categorical_powerset = powerset(interactions_list,2,3)
# Use the powerset to create new columns indicating the days since the last delivery for each combination of the item descriptions
stats_to_agg = ["mean","std","min","max","median","mad","skew"]
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        print(i)
        df_group = df[i+["delivery_time"]].groupby(i).agg(stats_to_agg).reset_index().round(2)
        df_group.columns = [str("_" + i[0] +"_" + i[1] + "_").join(col) for col in df_group.columns]
        df_group.rename(columns= {df_group.columns[0]:str(i[0]), df_group.columns[1]:str(i[1])},inplace=True)
        df_group.fillna(0,inplace=True)
        df = df.merge(df_group,on=i,how="left") 

Same but also add the 3x interaction between the categorical variables and the **delivery date**.

In [78]:
# Interactions between delivery time and categorical variables
interactions_list = ["item_id","brand_id","item_color","size","delivery_weekday","user_id"]
categorical_powerset = powerset(interactions_list,3,4)
# Use the powerset to create new columns indicating the days since the last delivery for each combination of the item descriptions
stats_to_agg = ["mean","std","min","max","median","mad","skew"]
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        print(i)
        df_group = df[i+["delivery_time"]].groupby(i).agg(stats_to_agg).reset_index().round(2)
        df_group.columns = [str("_" + i[0] +"_" + i[1] + "_" + i[2] + "_").join(col) for col in df_group.columns]
        df_group.rename(columns= {df_group.columns[0]:str(i[0]), df_group.columns[1]:str(i[1]),df_group.columns[2]:str(i[2])},inplace=True)
        df_group.fillna(0,inplace=True)
        df = df.merge(df_group,on=i,how="left") 

['item_id', 'item_color', 'size']
['item_id', 'item_color', 'delivery_weekday']
['item_id', 'item_color', 'user_id']
['item_id', 'size', 'delivery_weekday']
['item_id', 'size', 'user_id']
['item_id', 'delivery_weekday', 'user_id']
['brand_id', 'item_color', 'size']
['brand_id', 'item_color', 'delivery_weekday']
['brand_id', 'item_color', 'user_id']
['brand_id', 'size', 'delivery_weekday']
['brand_id', 'size', 'user_id']
['brand_id', 'delivery_weekday', 'user_id']
['item_color', 'size', 'delivery_weekday']
['item_color', 'size', 'user_id']
['item_color', 'delivery_weekday', 'user_id']
['size', 'delivery_weekday', 'user_id']


In [12]:
df = pd.read_csv("df_last_X3.csv")

Calculate the total items count and the cumulative sum of itmes ordered by each customer.

In [13]:
# Total items ordered by user
df_group = df[["user_id","user_state"]].groupby(["user_id"]).count().reset_index()
df_group.rename(columns= {"user_state":"user_id_total_items_ordered"},inplace=True)
# Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
df = df.join(df_group.set_index("user_id"),on="user_id",how="left")

In [14]:
# Cumulative sum of items ordered by user per order day
df_group = df[["user_id","user_state","order_date"]].groupby(["user_id","order_date"]).count().reset_index()
df_group.rename(columns= {"user_state":"user_id_total_items_ordered"},inplace=True)
df_group["user_id_total_items_ordered_cumsum"] = df_group.groupby(["user_id"])["user_id_total_items_ordered"].cumsum()
df_group.drop(columns=["user_id_total_items_ordered"],inplace=True)
df = df.join(df_group.set_index(["user_id","order_date"]),on=["user_id","order_date"],how="left")

In [15]:
# Total items ordered by user and interactions
interactions_list = ["item_id","brand_id","item_color","size"]
categorical_powerset = powerset(interactions_list,1,len(interactions_list)+1)
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        # print(i)
        df_group = df[["user_id","user_state"]+i].groupby(["user_id"]+i).count().reset_index()
        df_group.rename(columns= {"user_state":"user_id_"+ "_".join(i) + "_total_items_ordered"},inplace=True)
        # Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
        df = df.join(df_group.set_index(["user_id"]+i),on=["user_id"]+i,how="left")
        

In [16]:
# Cumulative sum of items ordered by user per order day
interactions_list = ["item_id","brand_id","item_color","size"]
categorical_powerset = powerset(interactions_list,1,len(interactions_list)+1)
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    i = list(column)
    if not all(x in i for x in exclude):
        print(i)
        df_group = df[["user_id","user_state","order_date"]+i].groupby(["user_id"]+i+["order_date"]).count().reset_index()
        df_group["user_state"] = df_group.groupby(["user_id"]+i)["user_state"].cumsum()
        df_group.rename(columns= {"user_state":"user_id_"+ "_".join(i) + "_cumsum_items_ordered"},inplace=True)
        # Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
        df = df.join(df_group.set_index(["user_id","order_date"]+i),on=["user_id","order_date"]+i,how="left")

['item_id']
['brand_id']
['item_color']
['size']
['item_id', 'item_color']
['item_id', 'size']
['brand_id', 'item_color']
['brand_id', 'size']
['item_color', 'size']
['item_id', 'item_color', 'size']
['brand_id', 'item_color', 'size']


Calculate the number of unique products ordered by each customer.

In [17]:
# n unique for user id
interactions_list = ["item_id","brand_id","item_color","size"]
categorical_powerset = powerset(interactions_list,0,len(interactions_list)-1)
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude) and current != ['item_id', 'item_color', 'size']:
        print(current)
        df_group = df[["user_id","item_id","size","item_color","brand_id"]].groupby(["user_id"]+current).nunique()
        df_group.columns = ["user_id_"+i+"_nunique_" +"_".join(current)  for i in df_group.columns]
        # Use join instead of merge to speed up the process as merge is slower CODEBASE INCLUDE !!!
        df = df.join(df_group,on=["user_id"]+current,how="left")

[]
['item_id']
['brand_id']
['item_color']
['size']
['item_id', 'item_color']
['item_id', 'size']
['brand_id', 'item_color']
['brand_id', 'size']
['item_color', 'size']


In [19]:
# df.to_csv("df_last_467.csv",index=False)

In [4]:
# df = pd.read_csv("df_last_467.csv")

In [72]:
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max,mode_item_id,mode_size,mode_brand_id,mode_item_color,delivery_time_item_id_mean,delivery_time_item_id_std,delivery_time_item_id_min,delivery_time_item_id_max,delivery_time_item_id_median,delivery_time_item_id_mad,delivery_time_size_mean,delivery_time_size_std,delivery_time_size_min,delivery_time_size_max,delivery_time_size_median,delivery_time_size_mad,delivery_time_brand_id_mean,delivery_time_brand_id_std,delivery_time_brand_id_min,delivery_time_brand_id_max,delivery_time_brand_id_median,delivery_time_brand_id_mad,delivery_time_item_color_mean,delivery_time_item_color_std,delivery_time_item_color_min,delivery_time_item_color_max,delivery_time_item_color_median,delivery_time_item_color_mad,delivery_time_user_id_mean,delivery_time_user_id_std,delivery_time_user_id_min,delivery_time_user_id_max,delivery_time_user_id_median,delivery_time_user_id_mad,days_since_last_order,days_since_last_order_user_id_mean,days_since_last_order_user_id_median,days_since_last_order_user_id_max,days_since_last_order_user_id_min,days_since_last_order_user_id_std,days_since_last_order_user_id_mad,days_since_last_order_user_id_skew,days_since_last_order_same_item_id,days_since_last_order_same_size,days_since_last_order_same_item_color,days_since_last_order_same_brand_id,days_since_last_order_same_item_id_size,days_since_last_order_same_item_id_item_color,days_since_last_order_same_size_item_color,days_since_last_order_same_size_brand_id,days_since_last_order_same_item_color_brand_id,days_since_last_order_same_item_id_size_item_color,days_since_last_order_same_size_item_color_brand_id,days_since_last_delivery,days_since_last_delivery_user_id_mean,days_since_last_delivery_user_id_median,days_since_last_delivery_user_id_max,days_since_last_delivery_user_id_min,days_since_last_delivery_user_id_st

Overall we managed to create a data frame with 467 Columns on which our final model will be trained.

In [34]:
# Instantiate the model class
Model = Model_class(df)

In [9]:
# Split the data into train and test sets
df_train,df_test, df_valid,k = Model.split_data(False)

In [76]:
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.05,True)

In [88]:
catboost_params_cpu = {'iterations': 175, 'learning_rate': 0.11, 'depth': 9, 'loss_function': 'Logloss', 'random_seed': 42,"l2_leaf_reg": 15,"max_ctr_complexity" : 3}
catboost_model_cpu, catboost_Y_pred_cpu, catboost_mae_cpu, catboost_Y_pred_proba_cpu = Model.catboost(df_train,df_test,catboost_params_cpu,1,cat_cols)
print("Catboost_cpu MAE on test set:", catboost_mae_cpu)

Catboost_cpu MAE on test set: 0.303


In [20]:
# concat df_train and df_test
df_train_whole = pd.concat([df_train,df_test],axis=0)

In [90]:
catboost_params_gpu = {'iterations': 175, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU","l2_leaf_reg": 15,"max_ctr_complexity" : 3}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

Catboost_gpu MAE on test set: 0.3025


In [91]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

Xgboost MAE on test set: 0.3121


In [78]:
mae_dict_1 = {"Catboost_native_CPU":catboost_mae_cpu,"Catboost_native_GPU":catboost_mae_gpu,"Xgboost":xgb_mae}
model_results_1 = pd.DataFrame(mae_dict_1,index=["MAE_4"])
model_results_1.sort_values(by="MAE_4",ascending=True,inplace=True,axis=1)
model_results_1 = model_results_1.transpose()
model_results_1.index.set_names(["Model"],inplace=True)
model_results_1.reset_index(inplace=True)
model_results_1

,Model,MAE_4
0,Catboost_native_GPU,0.3025
1,Catboost_native_CPU,0.3030
2,Xgboost,0.3121


In [79]:
model_performance = pd.read_csv("Model_Performances/Model_performance_1.csv")
model_performance

,Model,MAE_3
0,Catboost_native_GPU,0.3191
1,Catboost_native_CPU,0.3196
2,Xgboost,0.3258
3,Lightgbm_native,0.3301


In [80]:
model_results_whole = pd.merge(model_results_1,model_performance,on="Model",how="left")
model_results_whole["MAE_Improvement"] = model_results_whole["MAE_3"] - model_results_whole["MAE_4"]
model_results_whole = model_results_whole[["Model","MAE_3","MAE_4","MAE_Improvement"]]
model_results_whole

,Model,MAE_3,MAE_4,MAE_Improvement
0,Catboost_native_GPU,0.3191,0.3025,0.0166
1,Catboost_native_CPU,0.3196,0.3030,0.0166
2,Xgboost,0.3258,0.3121,0.0137


Through the feature engineering in this notebook, we managed to increase the performance of our **Catboost** models on the test set by **0.0166** in terms of MAE and the performance of the xgboost model by **0.0137**. The GPU variation of the Catboost algorithm is the best-performing model on the test set with an MAE of **0.3025**** Therefor we will be using this model for the final predictions.

In [81]:
# Training the model on the whole dataset and predicting on the validation set
catboost_params_gpu = {'iterations': 200, 'learning_rate': 0.11, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU","l2_leaf_reg": 15,"max_ctr_complexity" : 3}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train_whole,df_valid,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6699331	test: 0.6727814	best: 0.6727814 (0)	total: 1.35s	remaining: 4m 27s
1:	learn: 0.6513266	test: 0.6563626	best: 0.6563626 (1)	total: 2.78s	remaining: 4m 35s
2:	learn: 0.6369605	test: 0.6436451	best: 0.6436451 (2)	total: 4.17s	remaining: 4m 34s
3:	learn: 0.6258520	test: 0.6342483	best: 0.6342483 (3)	total: 5.53s	remaining: 4m 30s
4:	learn: 0.6164678	test: 0.6258636	best: 0.6258636 (4)	total: 6.88s	remaining: 4m 28s
5:	learn: 0.6086105	test: 0.6194124	best: 0.6194124 (5)	total: 8.1s	remaining: 4m 21s
6:	learn: 0.6022613	test: 0.6148477	best: 0.6148477 (6)	total: 9.4s	remaining: 4m 19s
7:	learn: 0.5969062	test: 0.6105342	best: 0.6105342 (7)	total: 10.7s	remaining: 4m 16s
8:	learn: 0.5923347	test: 0.6070752	best: 0.6070752 (8)	total: 11.9s	remaining: 4m 12s
9:	learn: 0.5876848	test: 0.6030622	best: 0.6030622 (9)	total: 13.2s	remaining: 4m 11s
10:	learn: 0.5839301	test: 0.5996968	best: 0.5996968 (10)	total: 14.5s	remaining: 4m 8s
11:	learn: 0.5806156	test: 0.5967347	best: 0

In [87]:
score = int(catboost_mae_gpu * 50078)
print("Points:",score)

Points: 14893


The model performed quite well on the validation set generating **14893 Points** using the scoring metric defined by the Data Mining Cup 2014 and placing us in the top 4 of the leaderboard. The Catboost GPU model proves to be quite capable of predicting the return of an item. Furthermore, it trains relatively fast on the GPU and can handle the large number of features we created, as well as handle the categorical variables without having to encode them ourselves as part of the data preprocessing.